In [1]:
import bayes3d as b
import numpy as np 


In [3]:
intrinsics = b.Intrinsics(
    height=360,
    width=480,
    fx=311.769, fy=311.769,
    cx=240.0, cy=180.0,
    near=0.1, far=100.0
)

In [4]:
b.setup_renderer(intrinsics)
# b.RENDERER.add_mesh_from_file("assets/cube.obj",mesh_name="cube_0", scaling_factor=0.5)

: 

: 